# Plastic glacier profiles on idealized bed

In [ ]:
import chakra
import sys
sys.path.insert(0, '/Users/lizz/Documents/GitHub/plastic-networks')
from SERMeQ.plastic_utilities_v2 import *
from SERMeQ.flowline_class_hierarchy import *

### Set up standard idealized bed profile

In [ ]:
# Bassis & Ultee bed profile
fls = chakra.bu_tidewater_bed()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
x = fls[-1].dis_on_line * fls[-1].dx_meter / 1000
ax.hlines(0, 0, 60, color='C0')
ax.plot(x, fls[-1].bed_h, color='k')
ax.set(ylim=(-350, 800), xlabel=('Distance along flowline [km]'), ylabel=('Altitude [m]'));
plt.show()

### Produce a SERMeQ-plastic surface profile

For now, we are going to call PlasticProfile, which produces a basic analytical surface profile for a quasi-steady-state plastic calving glacier.  This function was superseded by some fancier functionality in later SERMeQ